In [2]:
import os
from contextlib import contextmanager


@contextmanager
def set_dir(dir):
    curr_dir = os.getcwd()
    os.chdir(dir)
    print("Directory changed to given")
    yield
    os.chdir(curr_dir)
    print("Directory reverted back to original")


In [5]:
import re
import json
from os.path import join

with set_dir('../') as dir_update:
    from grape import Grammar
    from ParseTree import ParseTree
    from EvalTree import EvalSymRegTree, EvalKnapSackTree
    print("Modules imported")

Reg_GRAMMAR_FILE = 'simpleReg.bnf'
Reg_BNF_GRAMMAR = Grammar(
    join("../grammars", Reg_GRAMMAR_FILE))

knapsack_GRAMMAR_FILE = 'knapsack.bnf'
knapsack_BNF_GRAMMAR = Grammar(
    join("../grammars", knapsack_GRAMMAR_FILE))

with open('./knapsackWeights.json') as file:
    items = json.load(file)

with open('./simpleReg.json') as file:
    reg_meta = json.load(file)


Directory changed to given
Modules imported
Directory reverted back to original


In [68]:
from treelib import Tree, Node

T_NT = 'NT'
T_T = 'T'


class Token(object):

    def __init__(self, type, name, meta=None) -> None:
        self.type = type
        self.name = name
        self.meta = meta

    def __str__(self) -> str:
        '''
        String representation of class instance of the format
        'ClassName(token-type, token-name, meta-data)'.

        Examples: 
            Token(NT, '<expr>', None)
            Token(T, 'X1', {"value": 10, "weight": 50})
        '''
        return f"Token({self.type}, {self.name}, {self.meta})"

    def __repr__(self) -> str:
        return self.__str__()


class RegParseTree():

    def __init__(self, grammar, node_meta):
        self.branch_idx = 0
        self.start_rule = grammar.start_rule
        self.terminals = set(grammar.terminals)
        self.non_terminals = set(grammar.non_terminals)
        self.node_meta = node_meta
        self.tree = Tree()

    def node_name(self, token_name):
        return re.sub(r'<|>', '', token_name)

    def make_token(self, token_name):
        type = T_T if token_name in self.terminals else T_NT
        return Token(type, token_name.upper(), meta=self.node_meta.get(f'{token_name}'))

    def set_root(self):
        name = self.node_name(self.start_rule)
        data = self.make_token(name)
        self.tree.create_node(name.upper(),
                              name, data=data)
        self.to_expand = name

    def has_binary_op(self, tokens, chosen_prod=None):
        # TODO: in-case of inline function
        if chosen_prod and re.search(r'\(.+\)', chosen_prod):
            return True, None
        for i, t in enumerate(tokens):
            meta = self.node_meta.get(t)
            if meta and meta.get('type') == 'operator' and int(meta.get('arity')) == 2:
                return True, (i-1, i, i+1)
        return False, None

    def update_node_ids(self):
        if self.to_expand.find('_left_') != -1:
            old_id = self.tree.parent(self.to_expand).identifier
            new_idx = int(old_id.split('_')[-1]) + 2
            new_id = '_'.join(old_id.split('_')[:-1]) + f'_{new_idx}'
            self.tree.update_node(old_id, identifier=new_id)

        r_parent = self.tree.parent(self.to_expand).identifier
        grand_parent = self.tree.parent(r_parent)
        cut_off = None
        for i, sib in enumerate(self.tree.children(grand_parent)):
            if sib.identifier == r_parent:
                cut_off = i
                continue
            if cut_off:
                old_id = sib.identifier
                new_idx = int(old_id.split('_')[-1]) + 2
                new_id = '_'.join(old_id.split('_')[:-1]) + f'_{new_idx}'
                self.tree.update_node(old_id, identifier=new_id)

    def add_binary_operation(self, tokens, indices):
        l_idx, op_idx, r_idx = indices
        left_node = Node(tag=tokens[l_idx].upper(
        ), identifier=f'{tokens[l_idx]}_left_{self.branch_idx}', data=self.make_token(tokens[l_idx]))
        right_node = Node(tag=tokens[r_idx].upper(
        ), identifier=f'{tokens[r_idx]}_right_{self.branch_idx+2}', data=self.make_token(tokens[r_idx]))
        operator_node = Node(tag=tokens[op_idx].upper(
        ), identifier=f'{tokens[op_idx]}_{self.branch_idx+1}', data=self.make_token(tokens[op_idx]))
        self.tree.add_node(operator_node, parent=self.to_expand)
        self.tree.add_node(left_node, parent=operator_node.identifier)
        self.tree.add_node(right_node, parent=operator_node.identifier)

    def grow(self, chosen_prod):
        tokens = re.findall(
            r"<\w+>|{}".format('|'.join(self.terminals)), chosen_prod)
        tokens = [self.node_name(t) for t in tokens]
        flag, indices = self.has_binary_op(tokens)
        if flag:
            self.add_binary_operation(tokens, indices)
            if self.tree.root != self.to_expand: self.update_node_ids()
        elif self.to_expand.find('_op_') != -1:
            node_name = self.node_name(tokens[0])
            node_data = self.make_token(node_name)
            self.tree.update_node(self.to_expand, tag=node_name.upper(
            ), identfier=f'{node_name}_{self.branch_idx}', data=node_data)
        else:
            node_name = self.node_name(tokens[0])
            node_data = self.make_token(node_name)
            self.tree.create_node(node_name.upper(
            ), f'{node_name}_{self.branch_idx}', parent=self.to_expand, data=node_data)

    def set_expansion_node(self, idx, to_replace):
        self.branch_idx = idx
        node_name = self.node_name(to_replace)
        check_keys = [f'{node_name}_{idx}', f'{node_name}_left_{idx}', f'{node_name}_right_{idx}']
        for k in check_keys:
            if k in self.tree.nodes():
                self.to_expand = k

    def display(self, property=None):
        if property:
            return self.tree.show(data_property=property)
        return self.tree.show()


In [55]:
'np.tanh(np.sin(np.tanh(x[1])))MULTIPLYDIVIDEnp.tanh(<e>)'